<a href="https://colab.research.google.com/github/zeglami/AI_Algorithms/blob/main/MultinomialNB_NaiveBayesFromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import libraries
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string 

In [3]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [4]:
#Read csv files
#df=pd.read_csv('emails.csv')
df=pd.read_csv('/content/drive/My Drive/DataSets/Emails/emails.csv')
#Print the fisrt row
df.head(5)

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [5]:
#print the shap(nb of columns and rows)
df.shape
#get  the columns names
df.columns
#Check of duplicates and remove them
df.drop_duplicates(inplace=True)

In [7]:
#Show the new shape (nb of columns and rows)
df.shape
#show the number of missing(NaN,NAN,na) data for each column
df.isnull().sum()

text    0
spam    0
dtype: int64

In [8]:
#download the stopwords package
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
def custom_fit(data):
  unique_words=set()
  for each_sentence in data:
    for each_word in each_sentence.split(' '):
      if len(each_word)>2:
        unique_words.add(each_word)
  
  vocab={}
  for index,word in enumerate(sorted(list(unique_words))):
    vocab[word]=index
  return vocab

In [21]:
from collections import Counter
from scipy.sparse import csr_matrix
def custom_transform(data):
  vocab=custom_fit(data)
  row,col,val=[],[],[]
  for idx,sentence in enumerate(data):
    count_word=dict(Counter(sentence.split(' ')))
    for word,count in count_word.items():
      col_index=vocab.get(word)
      if len(word) >=2:
        col_index=vocab.get(word)
        if col_index is None :
          pass
        elif col_index >=0 :
          row.append(idx)
          col.append(col_index)
          val.append(count)
        
      
  return csr_matrix((val,(row,col)),shape=(len(data),len(vocab)))
message_bow1=custom_transform(df['text']).toarray()
print(message_bow1)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [34]:
class MultinomialNB():
  def __init__(self, alpha=1):
    self.alpha = alpha 
  def fit(self, X_train, y_train):
    m, n = X_train.shape
    self._classes = np.unique(y_train)
    n_classes = len(self._classes)

    # init: Prior & Likelihood
    self._priors = np.zeros(n_classes)
    self._likelihoods = np.zeros((n_classes, n))

    # Get Prior and Likelihood
    for idx, c in enumerate(self._classes):
        X_train_c = X_train[c == y_train]
        self._priors[idx] = X_train_c.shape[0] / m 
        self._likelihoods[idx, :] = ((X_train_c.sum(axis=0)) + self.alpha) / (np.sum(X_train_c.sum(axis=0) + self.alpha))
    return self
  def predict(self, X_test):
    return [self._predict(x_test) for x_test in X_test]
  def _predict(self, x_test):
    # Calculate posterior for each class
    posteriors = []
    for idx, c in enumerate(self._classes):
        prior_c = np.log(self._priors[idx])
        likelihoods_c = self.calc_likelihood(self._likelihoods[idx,:], x_test)
        posteriors_c = np.sum(likelihoods_c) + prior_c
        posteriors.append(posteriors_c)

    return self._classes[np.argmax(posteriors)]

  def calc_likelihood(self, cls_likeli, x_test):
    return np.log(cls_likeli) * x_test

  def score(self, X_test, y_test):
    y_pred = self.predict(X_test)
    return np.sum(y_pred == y_test)/len(y_test)

In [23]:
#split the data into 80% training and 20% testing
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(message_bow1,df['spam'],test_size=0.20,random_state=0)

In [24]:
#get the shape of the date
message_bow1.shape

(5695, 36767)

In [36]:
classifier=MultinomialNB().fit(X_train,y_train)
print(classifier.__dict__)


{'alpha': 1, '_classes': array([0, 1]), '_priors': array([0.75877963, 0.24122037]), '_likelihoods': array([[4.72191644e-04, 7.17616480e-06, 1.43523296e-06, ...,
        1.43523296e-06, 4.30569888e-06, 1.43523296e-06],
       [2.39827591e-03, 5.38777690e-04, 9.53588832e-06, ...,
        4.76794416e-06, 4.76794416e-06, 3.33756091e-05]])}


In [37]:
#Print the pedictions
print(classifier.predict(X_train))
#Print the actual values
print(y_train.values)

[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 

In [39]:
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
pred=classifier.predict(X_train)
print(classification_report(y_train,pred))
print()
print('Confusion matrix: \n',confusion_matrix(y_train,pred))
print()
print('Accuracy:',accuracy_score(y_train,pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3457
           1       0.99      1.00      0.99      1099

    accuracy                           1.00      4556
   macro avg       0.99      1.00      0.99      4556
weighted avg       1.00      1.00      1.00      4556


Confusion matrix: 
 [[3442   15]
 [   2 1097]]

Accuracy: 0.996268656716418


In [41]:
#Print the pedictions
print(classifier.predict(X_test))
#Print the actual values
print(y_test.values)

[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 

In [42]:
#Evalute the model on the test data set
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
pred=classifier.predict(X_test)
print(classification_report(y_test,pred))
print()
print('Confusion matrix: \n',confusion_matrix(y_test,pred))
print()
print('Accuracy:',accuracy_score(y_test,pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       870
           1       0.97      0.99      0.98       269

    accuracy                           0.99      1139
   macro avg       0.98      0.99      0.99      1139
weighted avg       0.99      0.99      0.99      1139


Confusion matrix: 
 [[862   8]
 [  2 267]]

Accuracy: 0.9912203687445127
